In [1]:
from ray_func import *
import time
import torch

def my_task():
    current_time = time.localtime()
    print(f'start task at {current_time.tm_hour:02d}:{current_time.tm_min:02d}:{current_time.tm_sec:02d}')
    time.sleep(5)
    a = torch.randn(1000, 1000)
    b = torch.randn(1000, 1000)
    c = torch.matmul(a, b)
    result = c.shape
    current_time = time.localtime()
    print(f'end task at {current_time.tm_hour:02d}:{current_time.tm_min:02d}:{current_time.tm_sec:02d}')
    return result

@ray.remote
class worker:
    def __init__(self, gpu_name):
        select_gpu(gpu_name)

    def task(self):
        return my_task()

def main():
    # sort GPUs by free memory
    # sorted_gpu_names = find_top_k_gpu(k=0)

    # Run 4 workers on top 4 GPUs simultaneously
    n_workers = 4
    free_memory_threshold = 1 # GB
    gpu_names = get_gpu_names()
    eligible_gpu = find_eligible_gpu(gpu_names, n_gpu=n_workers, free_memory_threshold=free_memory_threshold)
    print(f"Eligible GPUs: {eligible_gpu}")
    if len(eligible_gpu) == n_workers:
        node_names = [gpu.split('_GPU')[0] for gpu in eligible_gpu]
        workers = [worker.options(resources={f"node:{node_name}": 0.01}).remote(gpu) for gpu, node_name in zip(eligible_gpu, node_names)]
        results = ray.get([worker.task.remote() for worker in workers])
        print(results)
    # shutdown ray
    ray.shutdown()

if __name__ == "__main__":
    ray_address = 'auto'
    ray.init(address=ray_address, ignore_reinit_error=True)
    res = ray.cluster_resources()
    for k, v in res.items():
        print(k, v)

    main()

2025-03-18 16:09:31,157	INFO worker.py:1654 -- Connecting to existing Ray cluster at address: 10.11.140.31:6379...
2025-03-18 16:09:31,171	INFO worker.py:1832 -- Connected to Ray cluster. View the dashboard at http://127.0.0.1:8265 


accelerator_type:G 2.0
memory 585787361076.0
object_store_memory 255337440460.0
CPU 128.0
GPU 4.0
node:10.11.140.64 1.0
node:__internal_head__ 1.0
node:10.11.140.31 1.0
finding 4 GPUs with free memory greater than 1 GB
(CustomGPU pid=7296) selecting 10.11.140.31_GPU0
Found eligible GPU: 10.11.140.31_GPU0, Free memory: 23.37 GB
Found eligible GPU: 10.11.140.31_GPU1, Free memory: 23.42 GB
Found eligible GPU: 10.11.140.64_GPU0, Free memory: 7.60 GB
(CustomGPU pid=1958707, ip=10.11.140.64) selecting 10.11.140.64_GPU1 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
Found eligible GPU: 10.11.140.64_GPU1, Free memory: 23.43 GB
Eligible GPUs: ['10.11.140.31_GPU0', '10.11.140.31_GPU1', '10.11.140.64_GPU0', '10.11.140.64_GPU1']
(worker pid=7303) start task at 16:09:40
(worker pid=7303) end task at 16:09:4